# Turning Ebay Used Car Sales Data Into Insights

Are you in the market for a used car and want to know what to expect? 

Or maybe you are a dealership looking to gain insights into the German used car market.

Look no further than this project analyzing a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website. 

With 50,000 data points, this dataset provides a comprehensive look at the German used car market and includes information on vehicle type, price, transmission type, power, fuel type, brand, and more. 

I'll be cleaning the data and conducting analysis to gain valuable insights. 

Get ready to uncover the secrets of the German used car market and make informed decisions with the power of data.

In [177]:
# import libraries

import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [71]:
# read in data


autos = pd.read_excel('autos.xlsx')

In [72]:
autos.info()

autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   dateCrawled          50000 non-null  datetime64[ns]
 1   name                 50000 non-null  object        
 2   seller               49948 non-null  object        
 3   offerType            49948 non-null  object        
 4   price                49948 non-null  object        
 5   abtest               49948 non-null  object        
 6   vehicleType          44859 non-null  object        
 7   yearOfRegistration   49948 non-null  float64       
 8   gearbox              47270 non-null  object        
 9   powerPS              49948 non-null  float64       
 10  model                47193 non-null  object        
 11  odometer             49948 non-null  object        
 12  monthOfRegistration  49948 non-null  float64       
 13  fuelType             45470 non-

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004.0,manuell,158.0,andere,"150,000km",3.0,lpg,peugeot,nein,2016-03-26,0.0,79588.0,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997.0,automatik,286.0,7er,"150,000km",6.0,benzin,bmw,nein,2016-04-04,0.0,71034.0,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009.0,manuell,102.0,golf,"70,000km",7.0,benzin,volkswagen,nein,2016-03-26,0.0,35394.0,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007.0,automatik,71.0,fortwo,"70,000km",6.0,benzin,smart,nein,2016-03-12,0.0,33729.0,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003.0,manuell,0.0,focus,"150,000km",7.0,benzin,ford,nein,2016-04-01,0.0,39218.0,2016-04-01 14:38:50


In [73]:
autos.shape

(50000, 20)

### Observations on the dataset

Our dataset contains 50k rows and 20 columns. The majority of the columns are stored as strings (object). Except for "dateCrawled" and "name", all the columns contain some null values. A number of columns need to be recast as different data types. Column names use camelcase instead of Python's preferred snakecase.

In [74]:
print(autos.columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [75]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_pictures', 'postal_code',
       'last_seen']

In [76]:
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004.0,manuell,158.0,andere,"150,000km",3.0,lpg,peugeot,nein,2016-03-26,0.0,79588.0,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997.0,automatik,286.0,7er,"150,000km",6.0,benzin,bmw,nein,2016-04-04,0.0,71034.0,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009.0,manuell,102.0,golf,"70,000km",7.0,benzin,volkswagen,nein,2016-03-26,0.0,35394.0,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007.0,automatik,71.0,fortwo,"70,000km",6.0,benzin,smart,nein,2016-03-12,0.0,33729.0,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003.0,manuell,0.0,focus,"150,000km",7.0,benzin,ford,nein,2016-04-01,0.0,39218.0,2016-04-01 14:38:50


I changed several of the column names in our dataframe to enhance clarity and to conform with Python' preferred snake case notation.

## Exploring and Cleaning the Data

In [77]:
autos.describe(include = 'all', datetime_is_numeric = True)

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
count,50000,50000,49948,49948,49948,49948,44859,49948.000000,47270,49948.000000,47193,49948,49948.000000,45470,49948,40134,49948,49948.0,49948.000000,49948
unique,NaN,38749,2,2,2356,2,8,NaN,2,NaN,245,13,NaN,7,40,2,NaN,NaN,NaN,NaN
top,NaN,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,NaN,NaN,NaN,NaN
freq,NaN,78,49947,49947,1420,25734,12848,NaN,36950,NaN,4019,32393,NaN,30069,10677,35199,NaN,NaN,NaN,NaN
mean,2016-03-21 13:43:25.697280,NaN,NaN,NaN,NaN,NaN,NaN,2005.074317,NaN,116.381016,NaN,NaN,5.723212,NaN,NaN,NaN,2016-03-20 19:40:38.728277504,0.0,50814.064107,2016-03-30 04:05:10.498618624
min,2016-03-05 14:06:30,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,2015-06-11 00:00:00,0.0,1067.000000,2016-03-05 14:45:46
25%,2016-03-13 14:51:25.750000128,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,2016-03-13 00:00:00,0.0,30451.000000,2016-03-23 11:15:32.249999872
50%,2016-03-21 17:53:56,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,2016-03-21 00:00:00,0.0,49571.000000,2016-04-04 01:18:00.500000
75%,2016-03-29 14:36:47.750000128,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,2016-03-29 00:00:00,0.0,71526.500000,2016-04-06 10:45:28.249999872
max,2016-04-07 14:36:56,NaN,NaN,NaN,NaN,NaN,NaN,9999.000000,NaN,17700.000000,NaN,NaN,12.000000,NaN,NaN,NaN,2016-04-07 00:00:00,0.0,99998.000000,2016-04-07 14:58:50


Text columns where all or almost all values are the same can often be dropped as they don't have useful information for analysis.

In our case, seller and offer type are such columns.

We'll definitely need to take a closer look at the price, registration_year, registration_month, odometer, power_ps, postal_code and num_pictures columns.

In [78]:
autos["price"].value_counts()

$0         1420
$500        779
$1,500      732
$2,500      640
$1,200      639
           ... 
$414          1
$79,933       1
$5,198        1
$18,890       1
$16,995       1
Name: price, Length: 2356, dtype: int64

In [79]:
# drop rows where prices are null - probably due to problems at the data ingestion stage

bool_prices = autos["price"].notnull()

autos = autos[bool_prices]

In [ ]:
autos.shape # dropped rows

In [80]:
autos["price"] = (autos["price"]
                                  .str.replace("$", "")
                                  .str.replace(",","")
                                  .astype(int)
                 )


C:\Users\Raphael\AppData\Local\Temp\ipykernel_10180\1819823680.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace("$", "")


In [84]:
autos["odometer"].value_counts()

150,000km    32393
125,000km     5163
100,000km     2167
90,000km      1753
80,000km      1436
70,000km      1228
60,000km      1164
50,000km      1025
5,000km        967
40,000km       819
30,000km       788
20,000km       784
10,000km       261
Name: odometer, dtype: int64

In [85]:
autos["odometer"] = (autos["odometer"]
                     .str.replace(",","")
                     .str.replace("km","")
                    )

count      49948
unique        13
top       150000
freq       32393
Name: odometer, dtype: object

In [91]:
autos["odometer"] = autos["odometer"].astype(int)

In [96]:
autos.rename({"odometer":"odometer_km"}, axis = 1, inplace = True)

In [97]:
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004.0,manuell,158.0,andere,150000,3.0,lpg,peugeot,nein,2016-03-26,0.0,79588.0,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997.0,automatik,286.0,7er,150000,6.0,benzin,bmw,nein,2016-04-04,0.0,71034.0,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009.0,manuell,102.0,golf,70000,7.0,benzin,volkswagen,nein,2016-03-26,0.0,35394.0,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007.0,automatik,71.0,fortwo,70000,6.0,benzin,smart,nein,2016-03-12,0.0,33729.0,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003.0,manuell,0.0,focus,150000,7.0,benzin,ford,nein,2016-04-01,0.0,39218.0,2016-04-01 14:38:50


### Explore Odometer and Price columns

In [99]:
# drop the low interest columns seller, offer_type, num_pictures

autos.drop(["num_pictures", "seller", "offer_type"], axis = 1)

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004.0,manuell,158.0,andere,150000,3.0,lpg,peugeot,nein,2016-03-26,79588.0,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997.0,automatik,286.0,7er,150000,6.0,benzin,bmw,nein,2016-04-04,71034.0,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009.0,manuell,102.0,golf,70000,7.0,benzin,volkswagen,nein,2016-03-26,35394.0,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007.0,automatik,71.0,fortwo,70000,6.0,benzin,smart,nein,2016-03-12,33729.0,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003.0,manuell,0.0,focus,150000,7.0,benzin,ford,nein,2016-04-01,39218.0,2016-04-01 14:38:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2016-03-27 14:38:19,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,24900,control,limousine,2011.0,automatik,239.0,q5,100000,1.0,diesel,audi,nein,2016-03-27,82131.0,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,1980,control,cabrio,1996.0,manuell,75.0,astra,150000,5.0,benzin,opel,nein,2016-03-28,44807.0,2016-04-02 14:18:02
49997,2016-04-02 14:44:48,Fiat_500_C_1.2_Dualogic_Lounge,13200,test,cabrio,2014.0,automatik,69.0,500,5000,11.0,benzin,fiat,nein,2016-04-02,73430.0,2016-04-04 11:47:27
49998,2016-03-08 19:25:42,Audi_A3_2.0_TDI_Sportback_Ambition,22900,control,kombi,2013.0,manuell,150.0,a3,40000,11.0,diesel,audi,nein,2016-03-08,35683.0,2016-04-05 16:45:07


### Look for suspicious data

In [103]:
autos["odometer_km"].unique().shape

(13,)

In [104]:
autos["odometer_km"].describe()

count     49948.000000
mean     125735.765196
std       40040.125086
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

These values are probably created from a drop-down list. It looks to me as if the top odometer selection is 150k and over.

In [107]:
autos["odometer_km"].value_counts().sort_index(ascending = True)

5000        967
10000       261
20000       784
30000       788
40000       819
50000      1025
60000      1164
70000      1228
80000      1436
90000      1753
100000     2167
125000     5163
150000    32393
Name: odometer_km, dtype: int64

In [106]:
autos["odometer_km"].value_counts().head()

150000    32393
125000     5163
100000     2167
90000      1753
80000      1436
Name: odometer_km, dtype: int64

From an odometer perspective, all the used cars in our sample are quite heavily used. The median odometer value of 150k is indicative of that.

If we want to remove outliers, we can use the df[(df["col"] >= x ) & (df["col"] <= y )] syntax, but it's more readable to use df[df["col"].between(x,y)]

For the odometer_km column, there is not apparent outlier that I want to remove.

In [108]:
autos["price"].unique().shape

(2356,)

In [109]:
autos["price"].describe()

count    4.994800e+04
mean     9.845721e+03
std      4.813547e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [110]:
autos["price"].value_counts().sort_index(ascending = True)

0           1420
1            156
2              3
3              1
5              2
            ... 
10000000       1
11111111       2
12345678       3
27322222       1
99999999       1
Name: price, Length: 2356, dtype: int64

In [113]:
autos[autos["price"] != 99999999]

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004.0,manuell,158.0,andere,150000,3.0,lpg,peugeot,nein,2016-03-26,0.0,79588.0,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997.0,automatik,286.0,7er,150000,6.0,benzin,bmw,nein,2016-04-04,0.0,71034.0,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009.0,manuell,102.0,golf,70000,7.0,benzin,volkswagen,nein,2016-03-26,0.0,35394.0,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007.0,automatik,71.0,fortwo,70000,6.0,benzin,smart,nein,2016-03-12,0.0,33729.0,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003.0,manuell,0.0,focus,150000,7.0,benzin,ford,nein,2016-04-01,0.0,39218.0,2016-04-01 14:38:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2016-03-27 14:38:19,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,privat,Angebot,24900,control,limousine,2011.0,automatik,239.0,q5,100000,1.0,diesel,audi,nein,2016-03-27,0.0,82131.0,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,privat,Angebot,1980,control,cabrio,1996.0,manuell,75.0,astra,150000,5.0,benzin,opel,nein,2016-03-28,0.0,44807.0,2016-04-02 14:18:02
49997,2016-04-02 14:44:48,Fiat_500_C_1.2_Dualogic_Lounge,privat,Angebot,13200,test,cabrio,2014.0,automatik,69.0,500,5000,11.0,benzin,fiat,nein,2016-04-02,0.0,73430.0,2016-04-04 11:47:27
49998,2016-03-08 19:25:42,Audi_A3_2.0_TDI_Sportback_Ambition,privat,Angebot,22900,control,kombi,2013.0,manuell,150.0,a3,40000,11.0,diesel,audi,nein,2016-03-08,0.0,35683.0,2016-04-05 16:45:07


In [156]:
# access the indexes and values of the value_counts series
x_values = autos["price"].value_counts().values.tolist()

y_values = autos["price"].value_counts().index.tolist()
print(sorted(y_values))

[0, 1, 2, 3, 5, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 20, 25, 29, 30, 35, 40, 45, 47, 49, 50, 55, 59, 60, 65, 66, 70, 75, 79, 80, 89, 90, 99, 100, 110, 111, 115, 117, 120, 122, 125, 129, 130, 135, 139, 140, 145, 149, 150, 156, 160, 170, 173, 175, 179, 180, 185, 188, 190, 193, 195, 198, 199, 200, 205, 210, 215, 217, 219, 220, 222, 225, 230, 235, 240, 248, 249, 250, 251, 255, 260, 269, 270, 275, 277, 280, 285, 290, 295, 299, 300, 310, 320, 325, 329, 330, 333, 340, 349, 350, 356, 359, 360, 369, 370, 375, 378, 379, 380, 385, 388, 390, 395, 399, 400, 410, 414, 420, 425, 430, 435, 440, 444, 449, 450, 459, 460, 470, 475, 480, 485, 490, 495, 499, 500, 501, 510, 517, 520, 525, 530, 540, 549, 550, 554, 555, 560, 566, 570, 575, 578, 579, 580, 590, 595, 598, 599, 600, 606, 620, 625, 628, 630, 640, 644, 648, 649, 650, 655, 660, 666, 669, 670, 675, 679, 680, 686, 689, 690, 695, 699, 700, 710, 719, 720, 725, 729, 730, 740, 744, 745, 749, 750, 755, 760, 770, 777, 780, 785, 789, 790, 795, 799, 800, 810

It seems like 300k is a good cut-off for prices to consider. Upwards of 300k prices tend to increase to the improbable.

In [158]:
autos = autos[autos["price"].between(1, 301000)]

In [169]:
print(autos["price"].value_counts())

500      779
1500     732
2500     640
1000     639
1200     639
        ... 
414        1
79933      1
5198       1
18890      1
16995      1
Name: price, Length: 2343, dtype: int64


In [168]:
autos["price"].describe()

count     48512.000000
mean       5876.340452
std        8793.510699
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      299000.000000
Name: price, dtype: float64

After removing the outliers, the price column looks good now.

### Explore the date columns

In [173]:
autos.dtypes

date_crawled          datetime64[ns]
name                          object
seller                        object
offer_type                    object
price                          int32
ab_test                       object
vehicle_type                  object
registration_year            float64
gearbox                       object
power_ps                     float64
model                         object
odometer_km                    int32
registration_month           float64
fuel_type                     object
brand                         object
unrepaired_damage             object
ad_created            datetime64[ns]
num_pictures                 float64
postal_code                  float64
last_seen             datetime64[ns]
dtype: object

In [182]:
print(autos["date_crawled"].dt.date)

0        2016-03-26
1        2016-04-04
2        2016-03-26
3        2016-03-12
4        2016-04-01
            ...    
49995    2016-03-27
49996    2016-03-28
49997    2016-04-02
49998    2016-03-08
49999    2016-03-14
Name: date_crawled, Length: 48512, dtype: object


In [185]:
autos["date_crawled"].dt.date.value_counts(normalize = True, dropna = False).sort_index(ascending = True)

2016-03-05    0.025355
2016-03-06    0.013997
2016-03-07    0.035991
2016-03-08    0.033332
2016-03-09    0.033126
2016-03-10    0.032178
2016-03-11    0.032610
2016-03-12    0.036939
2016-03-13    0.015666
2016-03-14    0.036506
2016-03-15    0.034239
2016-03-16    0.029622
2016-03-17    0.031662
2016-03-18    0.012904
2016-03-19    0.034754
2016-03-20    0.037929
2016-03-21    0.037393
2016-03-22    0.033023
2016-03-23    0.032178
2016-03-24    0.029333
2016-03-25    0.031621
2016-03-26    0.032198
2016-03-27    0.031064
2016-03-28    0.034857
2016-03-29    0.034115
2016-03-30    0.033682
2016-03-31    0.031827
2016-04-01    0.033703
2016-04-02    0.035455
2016-04-03    0.038568
2016-04-04    0.036486
2016-04-05    0.013110
2016-04-06    0.003174
2016-04-07    0.001402
Name: date_crawled, dtype: float64